In [1]:
#%matplotlib inline
#%matplotlib notebook

In [2]:
import cytoflow as flow
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib
import scipy.stats as stats
import scipy as sp
from scipy.stats.mstats import gmean
import scipy.optimize as opt
import numpy as np
from numpy import median
matplotlib.rc('figure', dpi = 160)

ModuleNotFoundError: No module named 'cytoflow'

# Import samples and specify experimental condtions

sets the conditions of the tube

In [ ]:
inputs = [
    ["20210513_Tube_001_001.fcs",0,0,'WT'],
    ["20210513_Tube_002_002.fcs",1e-4,0,'LuxR'],
    ["20210513_Tube_003_003.fcs",1e-5,0,'LuxR'],
    ["20210513_Tube_004_004.fcs",1e-6,0,'LuxR'],
    ["20210513_Tube_005_005.fcs",1e-7,0,'LuxR'],
    ["20210513_Tube_006_006.fcs",1e-8,0,'LuxR'],
    ["20210513_Tube_007_007.fcs",1e-9,0,'LuxR'],
    ["20210513_Tube_008_008.fcs",1e-10,0,'LuxR'],
    ["20210513_Tube_009_009.fcs",0,0,'LuxR'],
    
    ["20210513_Tube_010_010.fcs",1e-4,0.005,'LuxR'],
    ["20210513_Tube_011_011.fcs",1e-5,0.005,'LuxR'],
    ["20210513_Tube_012_012.fcs",1e-6,0.005,'LuxR'],
    ["20210513_Tube_013_013.fcs",1e-7,0.005,'LuxR'],
    ["20210513_Tube_014_014.fcs",1e-8,0.005,'LuxR'],
    ["20210513_Tube_015_015.fcs",1e-9,0.005,'LuxR'],
    ["20210513_Tube_016_016.fcs",1e-10,0.005,'LuxR'],
    ["20210513_Tube_017_017.fcs",0,0.005,'LuxR'],
    
    ["20210513_Tube_018_018.fcs",1e-4,0.01,'LuxR'],
    ["20210513_Tube_019_019.fcs",1e-5,0.01,'LuxR'],
    ["20210513_Tube_020_020.fcs",1e-6,0.01,'LuxR'],
    ["20210513_Tube_021_021.fcs",1e-7,0.01,'LuxR'],
    ["20210513_Tube_022_022.fcs",1e-8,0.01,'LuxR'],
    ["20210513_Tube_023_023.fcs",1e-9,0.01,'LuxR'],
    ["20210513_Tube_024_024.fcs",1e-10,0.01,'LuxR'],
    ["20210513_Tube_025_025.fcs",0,0.01,'LuxR'],
    
    ["20210513_Tube_026_026.fcs",1e-4,0.05,'LuxR'],
    ["20210513_Tube_027_027.fcs",1e-5,0.05,'LuxR'],
    ["20210513_Tube_028_028.fcs",1e-6,0.05,'LuxR'],
    ["20210513_Tube_029_029.fcs",1e-7,0.05,'LuxR'],
    ["20210513_Tube_030_030.fcs",1e-8,0.05,'LuxR'],
    ["20210513_Tube_031_031.fcs",1e-9,0.05,'LuxR'],
    ["20210513_Tube_032_032.fcs",1e-10,0.05,'LuxR'],
    ["20210513_Tube_033_033.fcs",0,0.05,'LuxR'],
    
    ["20210513_Tube_034_034.fcs",1e-4,0.1,'LuxR'],
    ["20210513_Tube_035_035.fcs",1e-5,0.1,'LuxR'],
    ["20210513_Tube_036_036.fcs",1e-6,0.1,'LuxR'],
    ["20210513_Tube_037_037.fcs",1e-7,0.1,'LuxR'],
    ["20210513_Tube_038_038.fcs",1e-8,0.1,'LuxR'],
    ["20210513_Tube_039_039.fcs",1e-9,0.1,'LuxR'],
    ["20210513_Tube_040_040.fcs",1e-10,0.1,'LuxR'],
    ["20210513_Tube_041_041.fcs",0,0.1,'LuxR'],
    
    ["20210513_Tube_042_042.fcs",1e-4,0.5,'LuxR'],
    ["20210513_Tube_043_043.fcs",1e-5,0.5,'LuxR'],
    ["20210513_Tube_044_044.fcs",1e-6,0.5,'LuxR'],
    ["20210513_Tube_045_045.fcs",1e-7,0.5,'LuxR'],
    ["20210513_Tube_046_046.fcs",1e-8,0.5,'LuxR'],
    ["20210513_Tube_047_047.fcs",1e-9,0.5,'LuxR'],
    ["20210513_Tube_048_048.fcs",1e-10,0.5,'LuxR'],
    ["20210513_Tube_049_049.fcs",0,0.5,'LuxR']



]

       
tubes = []
for filename, ahl, iptg, family in inputs:
    tubes.append(flow.Tube(file = 'data/' + filename, conditions = {'AHL' : ahl, 'IPTG' : iptg, 'Family' : family}))
    
ex = flow.ImportOp(conditions = {'AHL' : 'float', 'IPTG' : 'float', 'Family' : 'str'},
                   tubes = tubes).apply()
       


In [ ]:
flow.set_default_scale("log")

# Plot FITC-A histograms for all IPTG and AHL induction conditions

In [ ]:

flow.HistogramView(channel = "FITC-A",
                   xfacet = "AHL",
                  yfacet = 'IPTG').plot(ex)

In [ ]:
flow.DensityView(xchannel = "FSC-A", 
                 ychannel = "SSC-A",
                 xscale = 'log',
                 yscale = 'log').plot(ex, min_quantile = 0.005)

# Singlet gating to remove doublets

In [ ]:

#channels = ['FSC-A','FSC-W','FSC-H','SSC-A','SSC-W','SSC-H']
#perm = combinations(channels,2)
FSC = [('FSC-A','FSC-W'),('FSC-A','FSC-H'),('FSC-W','FSC-H')]

#f, axes = plt.subplots(ncols=4,nrows=4)
#axs = axes.ravel()
for i,(x,y) in enumerate(FSC):  
    flow.DensityView(xchannel = x, 
                ychannel = y,
                xscale = 'log',
                yscale = 'log').plot(ex, min_quantile = 0.005)


In [ ]:
SSC = [('SSC-A','SSC-W'),('SSC-A','SSC-H'),('SSC-W','SSC-H')]
for i,(x,y) in enumerate(SSC):  
    flow.DensityView(xchannel = x, 
                ychannel = y,
                xscale = 'linear',
                yscale = 'linear').plot(ex, min_quantile = 0.005)

## Create and store singlet gate

saves the single data as ex2

In [ ]:
r = flow.Range2DOp(name = "Range2D",
                   xchannel = "SSC-W",
                   xlow = 55000,
                   xhigh = 80000,
                   ychannel = "SSC-H",
                   ylow = 100,
                   yhigh = 10000)
                   
rv = r.default_view(xscale ='log',yscale='log')

rv.plot(ex)
ex2 = r.apply(ex)

## Use a gaussian mixture model to further gate the population

In [ ]:
gm = flow.GaussianMixtureOp(name = "GM",
                            channels = ['FSC-A', 'SSC-A'],
                            scale = {'FSC-A' : 'log',
                                     'SSC-A' : 'log'},
                            num_components = 1,
                            sigma = 2)
gm.estimate(ex2)
ex_gm = gm.apply(ex2)
gm.default_view().plot(ex_gm, alpha = 0.02)

In [ ]:
gm3 = flow.GaussianMixtureOp(name = "GM3",
                            channels = ['FSC-A', 'Pacific Blue-A'],
                            scale = {'FSC-A' : 'log',
                                     'Pacific Blue-A' : 'log'},
                            num_components = 1,
                            sigma = 2)
gm3.estimate(ex_gm)
ex_gm2 = gm3.apply(ex_gm)
gm3.default_view().plot(ex_gm2, alpha = 0.02)

In [ ]:
df =ex_gm2.data

take the stuff from cytoflow and changing it to datatframe

# filter out events that don't fall within the gates

In [ ]:

df = df[(df.GM_1==True) & (df.GM3_1==True) & (df.Range2D==True) & (df['FITC-A'] >0)]
df

In [ ]:
df.columns 



In [ ]:
data = df[['FITC-A','AHL','IPTG','Family']]
data.insert(4,'Replicate',3)
data.to_pickle('../compiled-data/se/lux/LuxSE_3.pkl')
#uncomment these when you are ready to save replicates and compile replicate from other runs

In [ ]:
# Subtract WT background from samples
data = df[['FITC-A','AHL','IPTG','Family']]
data['FITC-A'] = data['FITC-A'] - data[data.Family=='WT']['FITC-A'].median()
data['FITC-A'] = data[data['FITC-A'] > 0]['FITC-A']
data = data[data.Family=='LuxR']
data.drop('Family',inplace=True,axis=1)

In [ ]:
data['FITC-A'].min()

In [ ]:
data

# Plot median FITC-A

use median or geometeric mean 

In [ ]:
mfi = data.groupby(['AHL','IPTG'])['FITC-A'].median().reset_index()

f, ax = plt.subplots(figsize=(8,6))
sns.pointplot(x='AHL',y='FITC-A',hue='IPTG',data=mfi)
f.savefig('Lux.png',dpi=600)

# Plot heatmaps showing absolute and relative induction

Raw signal 

In [ ]:
#LasMeans = Las.groupby(['C_ahl','C_iptg']).mean().reset_index()
sns.set(style="white")
#sns.set_style('Talk')
#LasTable = pd.pivot_table(LasMeans,values='FITC-A',index='C_iptg',columns='C_ahl')
f, ax = plt.subplots(figsize=(8,6))
#g = sns.heatmap(LasTable,vmin=0,vmax=100,xticklabels=[0,1e-10,1e-9,1e-7],yticklabels=[0,1e-3,1e-4,1e-5],ax=ax,square=True,cmap='Reds')



#f.savefig('LasHeatmap.png',dpi=600)

#mean['FITC-A']=mean['FITC-A']/mean['FITC-A'].max()*100

piv = mfi.pivot_table(index='IPTG', columns='AHL', values='FITC-A')
g = sns.heatmap(piv, annot=True, fmt=".1f",cmap='Blues')
cbar = g.collections[0].colorbar
#cbar.set_ticks([0, 25, 50, 75,100])
#cbar.set_ticklabels(['0%', '25%', '50%', '75%', '100%'])
ax.set_xlabel(r'[AHL] (M)')
ax.set_ylabel(r'[IPTG] (M)')
f.tight_layout()

Normalized to highest signal 

In [ ]:
f, ax = plt.subplots(figsize=(8,6))
# normalized
norm = mfi.copy()
norm['FITC-A']=norm['FITC-A']/norm['FITC-A'].max()*100

piv = norm.pivot_table(index='IPTG', columns='AHL', values='FITC-A')
g = sns.heatmap(piv, annot=True, fmt=".1f",cmap='Blues')
cbar = g.collections[0].colorbar
cbar.set_ticks([0, 25, 50, 75,100])
cbar.set_ticklabels(['0%', '25%', '50%', '75%', '100%'])
ax.set_xlabel(r'[AHL] (M)')
ax.set_ylabel(r'[IPTG] (M)')
f.tight_layout()

# Calculate the on/off ratio and plot

In [ ]:

R =[]
for i in mfi.IPTG.unique():
    tmp = mfi[mfi.IPTG ==i]['FITC-A'].max()/mfi[mfi.IPTG ==i]['FITC-A'].min()
    R.append([i,tmp])



In [ ]:

R = pd.DataFrame(R,columns=['IPTG','Induction Ratio'])
R

In [ ]:
f, ax  = plt.subplots()
sns.barplot(x='IPTG',y='Induction Ratio',data=R,palette='Blues')